In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 49.1 MB 1.4 MB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 344 kB 12.3 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595738 sha256=7cba1f733768e6e8e7e4c02a5c6bf34342ad46d48913190698805e09eeac61ed
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.2 MB 12.6 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.p

In [ ]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-yhk8gueh
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-yhk8gueh
     |████████████████████████████████| 132 kB 12.6 MB/s 
     |████████████████████████████████| 4.5 MB 48.3 MB/s 
     |████████████████████████████████| 4.4 MB 45.9 MB/s 
     |████████████████████████████████| 596 kB 40.6 MB/s 
     |████████████████████████████████| 6.6 MB 37.8 MB/s 
     |████████████████████████████████| 101 kB 6.2 MB/s 
     |████████████████████████████████| 79 kB 7.3 MB/s 
     |████████████████████████████████| 8.9 MB 42.7 MB/s 
     |████████████████████████████████| 127 kB 49.8 MB/s 
  Created wheel for kobert: filename=kobert-0.2.3-py3-none-any.whl size=15674 sha256=5cff35875479e67a1e820f08025cac4522998b4ccbce0be6cd0bb0df651c7506
  Stored in directory

In [ ]:
!pip uninstall folium 
!pip install folium==0.2.1

Found existing installation: folium 0.8.3
Uninstalling folium-0.8.3:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/folium-0.8.3.dist-info/*
    /usr/local/lib/python3.7/dist-packages/folium/*
Proceed (y/n)? y
  Successfully uninstalled folium-0.8.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 69 kB 5.6 MB/s 
  Created wheel for folium: filename=folium-0.2.1-py3-none-any.whl size=79808 sha256=68d6ec731dd09fffdd59516fe22f580460f59050e6ee3872f4d9767ab298f224
  Stored in directory: /root/.cache/pip/wheels/9a/f0/3a/3f79a6914ff5affaf50cabad60c9f4d565283283c97f0bdccf
Successfully built folium


In [ ]:
import torch
import pandas as pd
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
%$import numpy as np
from tqdm import tqdm, tqdm_notebook
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

UsageError: Line magic function `%$import` not found.


In [ ]:
device = torch.device("cuda:0")

In [ ]:
bertmodel, vocab = get_pytorch_kobert_model()

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


In [ ]:
dataset_train = nlp.data.TSVDataset('/content/reset_star_train.tsv', field_indices=[0,1], num_discard_samples=1)
dataset_test = nlp.data.TSVDataset('/content/reset_star_test.tsv', field_indices=[0,1], num_discard_samples=1)

In [ ]:
dataset_train[0]

['척테일러 같은 색상 같은 사이즈 세번째 구매인데 제가 알던 척테일러가 아니에요... 끈은 짧고 코는 크고 발은 불편하고...', '1']

In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 10
max_grad_norm = 1  
log_interval = 200
learning_rate =  5e-5

In [ ]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)
#BERTDataset 클래스 이용, TensorDataset으로 만들어주기
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=4)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=4)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=5, ##주의: 클래스 수 바꾸어 주세요!##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)

In [ ]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [ ]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} validation acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/25 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 1.640364170074463 train acc 0.234375
epoch 1 train acc 0.210625


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


...

epoch 9 validation acc 0.4375


  0%|          | 0/25 [00:00<?, ?it/s]

epoch 10 batch id 1 loss 0.7688061594963074 train acc 0.734375
epoch 10 train acc 0.73625


  0%|          | 0/7 [00:00<?, ?it/s]

epoch 10 validation acc 0.43080357142857145


# TEST

In [ ]:
new_test = nlp.data.TSVDataset('/content/reset_star_test.tsv', field_indices=[0,1], num_discard_samples=1)
test_set = BERTDataset(new_test , 0, 1, tok, max_len, True, False)
test_input = torch.utils.data.DataLoader(test_set, batch_size=1, num_workers=4)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_input)): 
  token_ids = token_ids.long().to(device) 
  segment_ids = segment_ids.long().to(device) 
  valid_length= valid_length 
  out = model(token_ids, valid_length, segment_ids)
  prediction = out.cpu().detach().numpy().argmax()
  print(batch_id, prediction)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


  0%|          | 0/400 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


0 1
1 0
2 0
3 1
4 3
5 4
6 2
7 4
8 3
9 1
10 3
11 2
12 0
13 1
14 3
15 4
16 4
17 4
18 4
19 1
20 4
21 3
22 1
23 1
24 2
25 1
26 3
27 2
28 0
29 1
30 0
31 2
32 0
33 2
34 0
35 1
36 3
37 3
38 0
39 2
40 4
41 0
42 1
43 1
44 0
45 3
46 2
47 4
48 2
49 0
50 2
51 0
52 1
53 1
54 4
55 2
56 3
57 0
58 3
59 2
60 2
61 0
62 4
63 2
64 4
65 1
66 2
67 1
68 1
69 1
70 2
71 3
72 4
73 4
74 4
75 4
76 3
77 2
78 1
79 1
80 4
81 1
82 4
83 2
84 1
85 4
86 3
87 4
88 0
89 2
90 2
91 0
92 2
93 2
94 4
95 1
96 1
97 1
98 2
99 2
100 3
101 3
102 1
103 1
104 3
105 4
106 1
107 4
108 0
109 4
110 4
111 3
112 0
113 4
114 3
115 2
116 0
117 0
118 4
119 0
120 3
121 4
122 4
123 1
124 2
125 0
126 4
127 4
128 0
129 0
130 4
131 4
132 0
133 3
134 4
135 2
136 4
137 1
138 2
139 1
140 2
141 4
142 3
143 1
144 0
145 3
146 3
147 2
148 1
149 4
150 4
151 4
152 3
153 4
154 1
155 3
156 4
157 3
158 3
159 1
160 4
161 3
162 2
163 0
164 2
165 4
166 2
167 3
168 1
169 0
170 1
171 0
172 2
173 4
174 3
175 3
176 3
177 1
178 4
179 1
180 3
181 0
182 1
183 1
184 2


In [ ]:
def test_calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    test_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return test_acc

In [ ]:
for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_input)): 
    token_ids = token_ids.long().to(device) 
    segment_ids = segment_ids.long().to(device) 
    valid_length= valid_length 
    label = label.long().to(device)
    out = model(token_ids, valid_length, segment_ids)
    prediction = out.cpu().detach().numpy().argmax()
    test_acc += calc_accuracy(out, label)
    print(batch_id, prediction+1)
print("epoch {} validation acc {}".format(e+1, test_acc / (batch_id+1)))


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


  0%|          | 0/400 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


0 2
1 1
2 1
3 2
4 4
5 5
6 3
7 5
8 4
9 2
10 4
11 3
12 1
13 2
14 4
15 5
16 5
17 5
18 5
19 2
20 5
21 4
22 2
23 2
24 3
25 2
26 4
27 3
28 1
29 2
30 1
31 3
32 1
33 3
34 1
35 2
36 4
37 4
38 1
39 3
40 5
41 1
42 2
43 2
44 1
45 4
46 3
47 5
48 3
49 1
50 3
51 1
52 2
53 2
54 5
55 3
56 4
57 1
58 4
59 3
60 3
61 1
62 5
63 3
64 5
65 2
66 3
67 2
68 2
69 2
70 3
71 4
72 5
73 5
74 5
75 5
76 4
77 3
78 2
79 2
80 5
81 2
82 5
83 3
84 2
85 5
86 4
87 5
88 1
89 3
90 3
91 1
92 3
93 3
94 5
95 2
96 2
97 2
98 3
99 3
100 4
101 4
102 2
103 2
104 4
105 5
106 2
107 5
108 1
109 5
110 5
111 4
112 1
113 5
114 4
115 3
116 1
117 1
118 5
119 1
120 4
121 5
122 5
123 2
124 3
125 1
126 5
127 5
128 1
129 1
130 5
131 5
132 1
133 4
134 5
135 3
136 5
137 2
138 3
139 2
140 3
141 5
142 4
143 2
144 1
145 4
146 4
147 3
148 2
149 5
150 5
151 5
152 4
153 5
154 2
155 4
156 5
157 4
158 4
159 2
160 5
161 4
162 3
163 1
164 3
165 5
166 3
167 4
168 2
169 1
170 2
171 1
172 3
173 5
174 4
175 4
176 4
177 2
178 5
179 2
180 4
181 1
182 2
183 2
184 3


In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# tokenizer = get_tokenizer()
# tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)



def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("1점")
            elif np.argmax(logits) == 1:
                test_eval.append("2점")
            elif np.argmax(logits) == 2:
                test_eval.append("3점")
            elif np.argmax(logits) == 3:
                test_eval.append("4점")
            elif np.argmax(logits) == 4:
                test_eval.append("5점")
            

        print(test_eval[0] + " 입니다.")

In [ ]:
predict(input('>> 리뷰 내용을 입력해주세요 : '))


>> 리뷰 내용을 입력해주세요 : 중국산 아니랄까봐... 저렴한만큼 합니다. 손잡이 마무리 엉망이에요... 이도 안맞게 박음질처리를 했네요....
1점 입니다.


In [ ]:
%

UsageError: Line magic function `%` not found.
